In [40]:
import h2o
import matplotlib

In [41]:
# Connect to a cluster
h2o.init()

H2O cluster uptime:,4 hours 18 minutes 37 seconds 174 milliseconds
H2O cluster version:,3.0.0.2
H2O cluster name:,anqi_fu
H2O cluster total nodes:,1
H2O cluster total memory:,1.78 GB
H2O cluster total cores:,8
H2O cluster allowed cores:,8
H2O cluster healthy:,True
H2O Connection ip:,127.0.0.1
H2O Connection port:,54321


In [42]:
# air_path = [h2o.locate("bigdata/laptop/airlines_all.05p.csv")]
air_path = [h2o.locate("bigdata/laptop/flights-nyc/flights14.csv.zip")]

# ----------

# 1- Load data - 1 row per flight.  Has columns showing the origin,
# destination, departure and arrival time, carrier information, and
# whether the flight was delayed.
print "Import and Parse airlines data"
data = h2o.import_frame(path=air_path)
data.describe()

Import and Parse airlines data

Parse Progress: [##################################################] 100%

Parsed 253,316 rows and 17 cols:



File1,/Users/anqi_fu/Documents/workspace/h2o-dev/bigdata/laptop/flights-nyc/flights14.csv.zip


Rows: 253,316 Cols: 17

Chunk compression summary:



chunk_type,count,count_percentage,size,size_percentage
C0L,48,14.860681,3.8 KB,0.06722641
C1,57,17.647058,745.9 KB,13.372172
C1N,66,20.433435,860.3 KB,15.423296
C2,152,47.058823,3.9 MB,71.13731



Frame distribution summary:



,size,number_of_rows,number_of_chunks_per_column,number_of_chunks
172.16.2.17:54321,5.4 MB,253316.0,19.0,323.0
mean,5.4 MB,253316.0,19.0,323.0
min,5.4 MB,253316.0,19.0,323.0
max,5.4 MB,253316.0,19.0,323.0
stddev,0 B,0.0,0.0,0.0
total,5.4 MB,253316.0,19.0,323.0



Column-by-Column Summary:



,year,month,day,dep_time,dep_delay,arr_time,arr_delay,cancelled,carrier,tailnum,flight,origin,dest,air_time,distance,hour,min
type,int,int,int,int,int,int,int,int,enum,enum,int,enum,enum,int,int,int,int
mins,2014.0,1.0,1.0,1.0,-112.0,1.0,-112.0,0.0,0.0,0.0,1.0,0.0,0.0,20.0,80.0,0.0,0.0
mean,2014.0,5.6385779027,15.8936979899,1338.23827946,12.4652647286,1494.3299002,8.14670214278,0.0,5.50707416823,1770.3521333,1840.81997584,0.988287356503,52.6592319474,156.722824456,1099.44527389,13.0634346034,31.8948191192
maxs,2014.0,10.0,31.0,2400.0,1498.0,2400.0,1494.0,0.0,13.0,3783.0,6443.0,2.0,108.0,706.0,4983.0,24.0,59.0
sigma,0.0,2.82086649965,8.76309681886,488.624226596,41.6487632369,533.817699585,45.749451513,0.0,4.04143332025,1123.41235681,1559.74865767,0.823529028955,29.0309561744,96.1297758678,752.982282058,4.89789057997,18.1386014
zero_count,0,0,0,0,12690,0,4331,253316,26302,8,0,87400,278,0,0,710,5269
missing_count,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [47]:
# ----------

# 2- Data exploration and munging. Generate scatter plots 
# of various columns and plot fitted GLM model.

# Function to fit a GLM model and plot the fitted (x,y) values
def scatter_plot(data, x, y, max_points = 1000, fit = True):
    if(fit):
        lr = h2o.glm(x = data[[x]], y = data[y], family = "gaussian")
        coeff = lr.coef()
    df = data[[x,y]]
    runif = df[y].runif()
    df_subset = df[runif < max_points/data.nrow()]
    df_py = data.frame(df.subset)   # TODO: How do I convert into Python lists?
    
    if(fit): h2o.remove(lr._key)
    
    matplotlib.pyplot.plot(df_py[x], df_py[y], "yo")
    matplotlib.pyplot.xlabel(x)
    matplotlib.pyplot.ylabel(y)
    
    if(fit):
        x_min = min(df_py[x])
        x_max = max(df_py[x])
        y_min = coeff[1] + coeff[2]*x_min
        y_max = coeff[2] + coeff[2]*x_max
        matplotlib.pyplot.plot([x_min, y_min], [x_max, y_max], "k-")
    matplotlib.pyplot.show()

scatter_plot(data, "Distance", "AirTime", fit = True)
scatter_plot(data, "Carrier", "ArrDelay", max_points = 5000, fit = False)

ValueError: Name Distance not in Frame

In [31]:
# Group flights by month
aggregates = {"Month": ["nrow", 0, "all"], "Cancelled": ["sum", 1, "all"]}
bpd = h2o.group_by(data, cols=["Month"], aggregates=aggregates)
bpd.show()
bpd.describe()
bpd.dim()

# Convert columns to factors
data["Year"]      = data["Year"]     .asfactor()
data["Month"]     = data["Month"]    .asfactor()
data["DayOfWeek"] = data["DayOfWeek"].asfactor()
data["Cancelled"] = data["Cancelled"].asfactor()

ValueError: Name Month not in Frame

In [39]:
# Calculate and plot travel time
hour1 = data["CRSArrTime"] / 100
mins1 = data["CRSArrTime"] % 100    # TODO: Need to implement modulus in Python
arrTime = hour1*60 + mins1

hour2 = data["CRSDepTime"] / 100
mins2 = data["CRSDepTime"] % 100
depTime = hour2*60 + mins2

# TODO: If-else method does not work in Python
travelTime = [x if x > 0 else None for x in (arrTime - depTime)]
data["TravelTime"] = travelTime
scatter_plot(data, "Distance", "TravelTime")

ValueError: Name CRSArrTime not in Frame

In [48]:
# Impute missing travel times and re-plot
h2o.impute(data = data, column = "Distance", by = ["Origin", "Dest"])
scatter_plot(data, "Distance", "TravelTime")

ValueError: Name Distance not in Frame

In [50]:
# ----------
# 3- Fit a model on train; using test as validation

# Create test/train split
data_split = h2o.split_frame(data, ratios = [0.75,0.25])
train = data_split[1]
test  = data_split[2]

# Set predictor and response variables
myY = "IsDepDelayed"
myX = ["Origin", "Dest", "Year", "UniqueCarrier", "DayOfWeek", "Month", "Distance", "FlightNum"]

# Simple GLM - Predict Delays
data_glm = h2o.glm(x           =train[myX],
                   y           =train[myY],
                   validation_x=test [myX],
                   validation_y=test [myY],
                   family      ="binomial",
                   standardize =True)

# Simple GBM
data_gbm = h2o.gbm(x              =train[myX],
                   y              =train[myY],
                   validation_x   =test [myX],
                   validation_y   =test [myY],
                   balance_classes=True,
                   ntrees         =3,
                   max_depth      =1,
                   distribution   ="bernoulli",
                   learn_rate     =0.1,
                   min_rows       =2)

# Complex GBM
data_gbm2 = h2o.gbm(x              =train[myX],
                    y              =train[myY],
                    validation_x   =test [myX],
                    validation_y   =test [myY],
                    balance_classes=True,
                    ntrees         =50,
                    max_depth      =5,
                    distribution   ="bernoulli",
                    learn_rate     =0.1,
                    min_rows       =2)

# Simple Random Forest
data_rf = h2o.random_forest(x              =train[myX],
                            y              =train[myY],
                            validation_x   =test [myX],
                            validation_y   =test [myY],
                            ntrees         =5,
                            max_depth      =2,
                            balance_classes=True)

# Complex Random Forest
data_rf2 = h2o.random_forest(x              =train[myX], 
                             y              =train[myY],
                             validation_x   =test [myX],
                             validation_y   =test [myY],
                             ntrees         =10,
                             max_depth      =5,
                             balance_classes=True)

# Deep Learning with 5 epochs
data_dl = h2o.deeplearning(x                   =train[myX],
                           y                   =train[myY],
                           validation_x        =test [myX],
                           validation_y        =test [myY],
                           hidden              =[10,10],
                           epochs              =5,
                           variable_importances=True,
                           balance_classes     =True,
                           loss                ="Automatic")

ValueError: Name Origin not in Frame

In [25]:
# Variable importances from each algorithm
abs(data_glm.coef_norm())   # Magnitude of normalized coefficients
data_gbm.varimp()
data_rf.varimp()

None


In [17]:
# Predict on test dataset with GBM model
best_pred = predict(data_gbm2, test)

NameError: name 'predict' is not defined